In [1]:
import collections
import copy
import datetime
import itertools
import sys

import json
import pathlib
import re
from pprint import pprint

#import nest_asyncio
#nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

from cibblbibbl import bbcode

In [2]:
show_Ids = True
G = cibblbibbl.CIBBL
Achievement = cibblbibbl.achievement.Achievement
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay
Season = cibblbibbl.season.Season
Tournament = cibblbibbl.tournament.handler.default.Tournament
NormalPlayer = cibblbibbl.player.NormalPlayer
StarPlayer = cibblbibbl.player.StarPlayer

In [3]:
w = cibblbibbl.phpwikicode

In [4]:
fname = "c:\\Users\\Szieberth\\Documents\\Repos\\cibblbibbl-data\\cibbl\\page\\achievements.20100926.wiki.txt"

In [5]:
G.init()

In [6]:
Se = max(G.seasons).prev
Se

Season('cibbl', 6, 4)

In [45]:
def trim_tournament_name(s):
    found = 0
    for i, c in enumerate(s):
        if c in "-–":
            found += 1
        if found == 2:
            return s[i+1:].strip()

In [8]:
def getcontent(a_cls, link_page=False):
    if isinstance(a_cls, list):
        logo_url, name, description, camelcasename = a_cls
    else:
        C = a_cls.defaultconfig_of_group(G)._data
        logo_url = C.get("logo_url", "")
        logo_url = f'https://fumbbl.com{logo_url}'
        name = C.get("phpwikiname", C["name"])
        description = C.get("phpwikidescription", C["description"])
        camelcasename = C.get("camelcasename")
    if link_page:
        pagename = f'CIBBL-Achievement-{camelcasename}'
        sn = w.url(pagename, name.lstrip("!"))
    else:
        sn = name
    return (
        w.url(logo_url) + w.br() + "\n"
        + w.b(sn) + w.br() + "\n"
        + w.i(description)
    )

In [56]:
teamac = {}
for k, cls in cibblbibbl.achievement.Achievement.registry.items():
    C = cls.defaultconfig_of_group(G)._data
    if not cls.__members__:
        continue
    if not C.get("description"):
        continue
    if cls.subject_typename == "Team":
        teamac[C["name"]] = cls
teamac["CabalVision Gold Partner"] = [
    "https://fumbbl.com/i/597778",
    "!CabalVision Gold Partner",
    "Awarded to a team which is a consecutive CIBBL member for the last 4 CIBBL Years (16 Seasons). Replaces Silver partnership. Worth 50 Prestige Points to the team as long as the team continously participates in every CIBBL events. Awarded since Year 5 Summer.",
    "CabalVisionGoldPartner",
]
teamac["CabalVision Silver Partner"] = [
    "https://fumbbl.com/i/597779",
    "!CabalVision Silver Partner",
    "Awarded to a team which is a consecutive CIBBL member for the last 2 CIBBL Years (8 Seasons). Worth 25 Prestige Points to the team as long as the team continously participates in every CIBBL events or until it becomes a Gold Partner. Awarded since Year 5 Summer.",
    "CabalVisionSilverPartner",
]

In [55]:
parts = []
gen = iter(sorted(teamac))
for k in gen:
    content1 = getcontent(teamac[k], True)
    try:
        content2 = getcontent(teamac[next(gen)], True)
    except StopIteration:
        content2 = ""
    parts.append(w.dualcolumn(content1, content2))
teamaccontent = "\n".join(parts)

In [73]:
playerac = collections.defaultdict(set)
for k, cls in cibblbibbl.achievement.Achievement.registry.items():
    if not cls.__members__:
        continue
    D = cls.defaultconfig_of_group(G)
    C = D._data
    bracket = C.get("bracket")
    if not bracket:
        continue
    playerac[bracket].add(cls)

In [74]:
bracketorder = [
    "Development",
    "Performance",
    "Best Players of Winter Tournaments",
    "Best Players of Spring Tournaments",
    "Best Players of Summer Tournaments",
    "Best Players of Autumn Tournaments",
    "Other Best Players",
]

In [83]:
parts = []
for bracket in bracketorder:
    parts.append(f'{w.br()}\n!!{bracket}')
    bracketparts = []
    clss0 = playerac[bracket]
    clss1 = sorted(clss0, key=lambda c: c.defaultconfig_of_group(G)._data.get("bracketsortkey", c.defaultconfig_of_group(G)._data["name"])) 
    gen = iter(clss1)
    for cls in gen:
        content1 = getcontent(cls, True)
        try:
            content2 = getcontent(next(gen), True)
        except StopIteration:
            content2 = ""
        bracketparts.append(w.dualcolumn(content1, content2))
    bracketcontent = "\n".join(bracketparts)
    parts.append(bracketcontent)
playeraccontent = "\n\n\n\n\n".join(parts)

In [84]:
parts = [
    w.center(w.url("https://fumbbl.com/i/596322")),
    "\n\n",
    w.br(),
    f'!!!Team Achievements',
    w.br(),
    "\n\n\n",
    teamaccontent,
    "\n\n",
    w.br(),
    f'!!!Player Achievements',
    "\n\n\n",
    playeraccontent,
]
content = "\n".join(parts)

In [85]:
#cibblbibbl.session.helppage.edit("CIBBL-Achievements", content)

<Response [200]>

In [62]:
name = 'Crushing Victory'
cls = teamac[name]
C = cls.defaultconfig_of_group(G)._data
aname = C.get("camelcasename",cls.__name__.split("_")[1])
pagename = f'CIBBL-Achievement-{aname}'
m0 = cls.__members__.values()
m1 = sorted(m0, key=lambda a: a.match.Id)
header = ["Tournament", "Match", "Team", "Roster", "Coach", "Prestige"]
header = [w.b(s) for s in header]
align = "CCLLLR"
rows = []
for a in m1:
    date = f'Y{a.season.year.nr} {a.season.name}'
    tournamentname = trim_tournament_name(a.tournament.shortname)
    tournament = w.tournament(a.tournament, tournamentname)
    stournament = f'{date} – {w.i(tournament)}'
    match = w.url(f'https://fumbbl.com/p/match?id={a.match.Id}', "-".join(str(x) for x in sorted(a.match.scores().values(), reverse=True)))
    team = w.url(f'https://fumbbl.com/p/team?team_id={a.subject.Id}', a.subject.name)
    roster = a.subject.roster_name
    coach = w.url(f'https://fumbbl.com/~{a.subject.coach_name}', a.subject.coach_name)
    prestige = a.prestige(season=Se)
    rows.append([stournament, match, team, roster, coach, prestige])
stable = w.table(rows, align=align, header=header)
parts = [
    "{center\n[https://fumbbl.com/i/646337]%%%\n}",
    w.dualcolumn(getcontent(c), ""),
    stable,
    "",
]
content = f'\n\n{w.br()}\n\n'.join(parts)
#print(content)
#cibblbibbl.session.helppage.edit(pagename, content)

In [86]:
print("-"*50)
for name in ('Obsidian Chalice', 'Pearl Chalice', "Winter's Glory"):
    cls = teamac[name]
    C = cls.defaultconfig_of_group(G)._data
    aname = C.get("camelcasename",cls.__name__.split("_")[1])
    pagename = f'CIBBL-Achievement-{aname}'
    m0 = cls.__members__.values()
    m1 = sorted(m0, key=lambda a: (a.season.year.nr, a.season.nr, int(a.tournament.Id), a.subject))
    # TODO tournament importance instead of id
    header = ["Tournament", "Team", "Roster", "Coach"]
    header = [w.b(s) for s in header]
    rows = []
    for a in m1:
        date = f'Y{a.season.year.nr} {a.season.name}'
        tournamentname = trim_tournament_name(a.tournament.shortname)
        tournament = w.tournament(a.tournament, tournamentname)
        stournament = f'{date} – {w.i(tournament)}'
        team = w.url(f'https://fumbbl.com/p/team?team_id={a.subject.Id}', a.subject.name)
        roster = a.subject.roster_name
        coach = w.url(f'https://fumbbl.com/~{a.subject.coach_name}', a.subject.coach_name)
        rows.append([stournament, team, roster, coach])
    stable = w.table(rows, header=header)
    parts = [
        "{center\n[https://fumbbl.com/i/646337]%%%\n}",
        w.dualcolumn(getcontent(cls), ""),
        stable,
        "",
    ]
    content = f'\n\n{w.br()}\n\n'.join(parts)
    print(pagename)
    print("-"*50)
    #print(content)
    print("-"*50)
    #cibblbibbl.session.helppage.edit(pagename, content) # uncomment to publish

--------------------------------------------------
CIBBL-Achievement-ObsidianChalice
--------------------------------------------------
--------------------------------------------------
CIBBL-Achievement-PearlChalice
--------------------------------------------------
--------------------------------------------------
CIBBL-Achievement-WintersGlory
--------------------------------------------------
--------------------------------------------------


In [64]:
partners = {
    "gold": cibblbibbl.helper.keydefaultdict((lambda k: k.gold_partner_teams())),
    "silver": cibblbibbl.helper.keydefaultdict((lambda k: k.silver_partner_teams())),
}
newpartners = {"gold": {}, "silver": {}}
endedpartners = {"gold": {}, "silver": {}}
seasons = sorted(se for se in G.seasons if se <= Se)
for S1 in seasons:
    g1 = partners["gold"][S1]
    s1 = partners["silver"][S1]
    S0 = S1.prev
    if S0:
        g0 = partners["gold"][S0]
        s0 = partners["silver"][S0]
    else:
        g0, s0 = set(), set()
    newpartners["gold"][S1] = g1-g0
    newpartners["silver"][S1] = s1-s0
    endedpartners["gold"][S1] = g0-g1
    endedpartners["silver"][S1] = s0-s1|newpartners["gold"][S1]

In [65]:
partnerships = {"gold": [], "silver": []}
lastp = {"gold": {}, "silver": {}}
for k in ("gold", "silver"):
    for S1 in seasons:
        new = newpartners[k][S1]
        for Te in new:
            lastp[k][Te] = S1
        ended = endedpartners[k][S1]
        for Te in ended:
            try:
                S0 = lastp[k][Te]
            except KeyError: # got gold immediately
                continue
            partnerships[k].append([Te, S0, S1])
            del lastp[k][Te]
    else:
        for Te, S0 in lastp[k].items():
            partnerships[k].append([Te, S0, None])
for k in ("gold", "silver"):
    L = partnerships[k]
    L.sort(key=lambda t: (t[1], t[0].name))
#partnerships

In [72]:
print("-"*50)
for k in ("gold", "silver"):
    name = f'CabalVision {k.title()} Partner'
    t = teamac[name]
    aname = "".join(name.split())
    pagename = f'CIBBL-Achievement-{aname}'
    header = ["Team", "Roster", "Coach", "Started", "Ended"]
    header = [w.b(s) for s in header]
    L = partnerships[k]
    rows = []
    for Te, S0, S1 in L:
        started = f'Y{S0.year.nr} {S0.name}'
        if S1:
            ended = f'Y{S1.year.nr} {S1.name}'
        else:
            ended = " "
        team = w.url(f'https://fumbbl.com/p/team?team_id={Te.Id}', Te.name)
        roster = Te.roster_name
        coach = w.url(f'https://fumbbl.com/~{Te.coach_name}', Te.coach_name)
        rows.append([team, roster, coach, started, ended])
    stable = w.table(rows, header=header)
    parts = [
        "{center\n[https://fumbbl.com/i/646337]%%%\n}",
        w.dualcolumn(getcontent(t), ""),
        stable,
        "",
    ]
    content = f'\n\n{w.br()}\n\n'.join(parts)
    print(pagename)
    print("-"*50)
    #print(content)
    print("-"*50)
    #cibblbibbl.session.helppage.edit(pagename, content) # uncomment to publish

--------------------------------------------------
CIBBL-Achievement-CabalVisionGoldPartner
--------------------------------------------------
--------------------------------------------------
CIBBL-Achievement-CabalVisionSilverPartner
--------------------------------------------------
--------------------------------------------------


In [68]:
et = cibblbibbl.achievement.exporttools

In [70]:
deadstars = set()
for Mu in G.matchups:
    if Se < Mu.season:
        continue
    for teamId, playerId, dpp in Mu.iterdead():
        if playerId.startswith("STAR-"):
            deadstars.add(player(playerId))
deadstars

{StarPlayer('STAR-Angus Porterhouse'),
 StarPlayer('STAR-Bilerot Vomitflesh'),
 StarPlayer('STAR-Flint Churnblade'),
 StarPlayer('STAR-Glart Smashrip Jr.'),
 StarPlayer('STAR-Gorlem Goreblade'),
 StarPlayer("STAR-Grak'Ng'Grak Gorthag"),
 StarPlayer('STAR-Grashnak Blackhoof'),
 StarPlayer('STAR-Gregor Miessen'),
 StarPlayer('STAR-Griff Oberwald'),
 StarPlayer('STAR-Hack Enslash'),
 StarPlayer('STAR-Helmut Wulf'),
 StarPlayer('STAR-Hoshi Komi'),
 StarPlayer('STAR-Lewdgrip Whiparm'),
 StarPlayer('STAR-Lord Borak the Despoiler'),
 StarPlayer('STAR-Max Spleenripper'),
 StarPlayer('STAR-Mighty Zug'),
 StarPlayer("STAR-Morg 'n' Thorg"),
 StarPlayer('STAR-Ritter Von Baum'),
 StarPlayer('STAR-Setekh'),
 StarPlayer("STAR-Siggi 'The Boot' Schuster"),
 StarPlayer('STAR-Sinnedbad'),
 StarPlayer('STAR-Skitter Stab-Stab'),
 StarPlayer('STAR-Wilhelm Chaney'),
 StarPlayer('STAR-Zara the Slayer'),
 StarPlayer('STAR-Ziggi Abschuss')}

In [22]:
playerstatuses = cibblbibbl.helper.keydefaultdict(lambda Pl: (Pl.status if hasattr(Pl, "status") else ""))

In [24]:
playerpositons = cibblbibbl.helper.keydefaultdict(lambda Pl: Pl.position)
playerpositonnames = cibblbibbl.helper.keydefaultdict(lambda Pl: (playerpositons[Pl].name if hasattr(playerpositons[Pl], "name") else ""))

In [81]:
print("-"*50)
for bracket, clss in playerac.items():
    for cls in clss:
        D = cls.defaultconfig_of_group(G)
        C = D._data
        aname = C.get("camelcasename",cls.__name__.split("_")[1])
        pagename = f'CIBBL-Achievement-{aname}'
        print(pagename)
        print("-"*50)
        if aname in {"AerodynamicAim", "MeanScoringMachine", "Perfection", "TripleX"}:
            m0 = cls.__members__.values()
            rows = []
            header = ["Tournament", "Match", "Player", "Position", "Team", "Prestige", "Status"]
            header = [w.b(s) for s in header]
            align = "CCCCCRC"
            rows = []
            for a in m0:
                if "transferred" in a["status"]:
                    continue
                date = f'Y{a.season.year.nr} {a.season.name}'
                date = f'Y{a.season.year.nr} {a.season.name}'
                tournamentname = trim_tournament_name(a.tournament.shortname)
                tournament = w.tournament(a.tournament, tournamentname)
                stournament = f'{date} – {w.i(tournament)}'
                team = et.team(a)
                scores = copy.copy(a.match.scores())
                teamofmatch = et.teamofmatch(a)
                oppoteam = et.oppoteam(a, team_=teamofmatch)
                if teamofmatch is None:
                    coach = ""
                    sscores = "?"              
                else:
                    coach = w.url(f'https://fumbbl.com/~{teamofmatch.coach_name}', teamofmatch.coach_name)
                    try:
                        sscores = f'{scores[teamofmatch]}-{scores[oppoteam]}'              
                    except KeyError:
                        sscores = ""
                if team == teamofmatch and hasattr(teamofmatch, "roster_name"):
                    roster = teamofmatch.roster_name
                elif team == "Star Player":
                    roster = team
                    team = teamofmatch
                else:
                    roster = ""
                match = w.url(f'https://fumbbl.com/p/match?id={a.match.Id}', sscores)
                prestige = a.prestige(season=Se)
                player_ = a.subject
                position = playerpositonnames[player_]
                status = playerstatuses[player_]
                if status == "Dead" and player_.nexts:
                    status = "Transf."
                elif player_ in deadstars:
                    status = "Dead"
                splayer = f'{w.player(player_)}'
                sroster = f'{roster}{w.br()}{w.i(position)}'
                steam = f'{w.team(team)}{w.br()}{w.i("(" + coach + ")")}'
                rows.append([stournament, match, splayer, sroster, steam, prestige, status]) 
            stable = w.table(rows, header=header, align=align)
            parts = [
                "{center\n[https://fumbbl.com/i/646337]%%%\n}",
                w.dualcolumn(getcontent(cls), ""),
                stable,
                "",
            ]
            content = f'\n\n{w.br()}\n\n'.join(parts)
            #print(content)
            print("-"*50)
            #cibblbibbl.session.helppage.edit(pagename, content) # uncomment to publish

--------------------------------------------------
CIBBL-Achievement-MeanScoringMachine
--------------------------------------------------
--------------------------------------------------
CIBBL-Achievement-FameKiller
--------------------------------------------------
CIBBL-Achievement-AerodynamicAim
--------------------------------------------------
--------------------------------------------------
CIBBL-Achievement-TripleX
--------------------------------------------------
--------------------------------------------------
CIBBL-Achievement-BewareSupremeKiller
--------------------------------------------------
CIBBL-Achievement-Perfection
--------------------------------------------------
--------------------------------------------------
CIBBL-Achievement-TargetEliminated
--------------------------------------------------
CIBBL-Achievement-ChildOfWinter
--------------------------------------------------
CIBBL-Achievement-SouthlandSalvation
-----------------------------------------

In [90]:
print("-"*50)
for bracket, clss in playerac.items():
    for cls in clss:
        D = cls.defaultconfig_of_group(G)
        C = D._data
        aname = C.get("camelcasename",cls.__name__.split("_")[1])
        pagename = f'CIBBL-Achievement-{aname}'
        print(pagename)
        print("-"*50)
        if aname in {"StarPlayer", "SuperStarPlayer", "MegaStarPlayer", "Legend"}:
            m0 = cls.__members__.values()
            m1 = sorted(m0, key=lambda a: (a.season.year.nr, a.season.nr, int(a.tournament.Id), a.subject))
            rows = []
            header = ["Tournament", "Match", "Player", "Position", "Team", "Prestige", "Status"]
            header = [w.b(s) for s in header]
            align = "CCCCCRC"
            rows = []
            for a in m0:
                if "transferred" in a["status"]:
                    continue
                if "postponed" in a["status"]:
                    continue
                date = f'Y{a.season.year.nr} {a.season.name}'
                date = f'Y{a.season.year.nr} {a.season.name}'
                tournamentname = trim_tournament_name(a.tournament.shortname)
                tournament = w.tournament(a.tournament, tournamentname)
                stournament = f'{date}{w.br()}{w.i(tournament)}'
                team = et.team(a)
                scores = copy.copy(a.match.scores())
                teamofmatch = et.teamofmatch(a)
                oppoteam = et.oppoteam(a, team_=teamofmatch)
                if teamofmatch is None:
                    coach = ""
                    sscores = "?"              
                else:
                    coach = w.url(f'https://fumbbl.com/~{teamofmatch.coach_name}', teamofmatch.coach_name)
                    try:
                        sscores = f'{scores[teamofmatch]}-{scores[oppoteam]}'              
                    except KeyError:
                        sscores = ""
                if team == teamofmatch and hasattr(teamofmatch, "roster_name"):
                    roster = teamofmatch.roster_name
                elif team == "Star Player":
                    roster = team
                    team = teamofmatch
                else:
                    roster = ""
                match = w.url(f'https://fumbbl.com/p/match?id={a.match.Id}', sscores)
                prestige = a.prestige(season=Se)
                player_ = a.subject
                position = playerpositonnames[player_]
                status = playerstatuses[player_]
                if status == "Dead" and player_.nexts:
                    status = "Transf."
                elif player_ in deadstars:
                    status = "Dead"
                splayer = f'{w.player(player_)}'
                sroster = f'{roster}{w.br()}{w.i(position)}'
                steam = f'{w.team(team)}{w.br()}{w.i("(" + coach + ")")}'
                rows.append([stournament, match, splayer, sroster, steam, prestige, status]) 
            stable = w.table(rows, header=header, align=align)
            parts = [
                "{center\n[https://fumbbl.com/i/646337]%%%\n}",
                w.dualcolumn(getcontent(cls), ""),
                stable,
                "",
            ]
            content = f'\n\n{w.br()}\n\n'.join(parts)
            #print(content)
            print("-"*50)
            cibblbibbl.session.helppage.edit(pagename, content) # uncomment to publish

--------------------------------------------------
CIBBL-Achievement-MeanScoringMachine
--------------------------------------------------
CIBBL-Achievement-FameKiller
--------------------------------------------------
CIBBL-Achievement-AerodynamicAim
--------------------------------------------------
CIBBL-Achievement-TripleX
--------------------------------------------------
CIBBL-Achievement-BewareSupremeKiller
--------------------------------------------------
CIBBL-Achievement-Perfection
--------------------------------------------------
CIBBL-Achievement-TargetEliminated
--------------------------------------------------
CIBBL-Achievement-ChildOfWinter
--------------------------------------------------
CIBBL-Achievement-SouthlandSalvation
--------------------------------------------------
CIBBL-Achievement-BugmansFinest
--------------------------------------------------
CIBBL-Achievement-CavernClimber
--------------------------------------------------
CIBBL-Achievement-TimelessDi

In [91]:
Pl = player("12871778")

In [98]:
for a in Pl.achievements:
    print(a)
    print(a["status"])
    print(a.prestige())

Child of Winter([47898] CIBBL - Y6, Winter - Friendlies (Low-Mid), [12871778] Coke en stock)
awarded
0
Super Star Player([51681] CIBBL - Y6, Autumn - Iron Division, [12871778] Coke en stock, Match(4222368))
awarded
10
Forged Greatness([51681] CIBBL - Y6, Autumn - Iron Division, [12871778] Coke en stock)
awarded
10
Mean Scoring Machine([47502] CIBBL - Y5, Autumn - Tin Division, [12871778] Coke en stock, Match(4105540))
awarded
10
Terrific Might of Tin([47502] CIBBL - Y5, Autumn - Tin Division, [12871778] Coke en stock)
awarded
10
Star Player([47898] CIBBL - Y6, Winter - Friendlies (Low-Mid), [12871778] Coke en stock, Match(4119033))
postpone awarded
0


In [100]:
Pl = player("11713837")
for a in sorted(Pl.achievements):
    print(a)
    print(a["status"])
    print(a.prestige())

Youthful Vibrance([42748] CIBBL - Year One, Autumn - Youngbloods III, [11713837] Carol I)
awarded
10
Star Player([43152] CIBBL - Year Two, Summer - Conjoined Blue Eagle II, [11713837] Carol I, Match(3921666))
awarded
10
Triple X([43152] CIBBL - Year Two, Summer - Conjoined Blue Eagle II, [11713837] Carol I, Match(3911591))
awarded
0
Bronze Excellence([43565] CIBBL - Year Two, Autumn - Bronze Division, [11713837] Carol I)
awarded
10
Super Star Player([43565] CIBBL - Year Two, Autumn - Bronze Division, [11713837] Carol I, Match(3931160))
awarded
10
Triple X([43565] CIBBL - Year Two, Autumn - Bronze Division, [11713837] Carol I, Match(3924129))
awarded
0
Triple X([43565] CIBBL - Year Two, Autumn - Bronze Division, [11713837] Carol I, Match(3928115))
awarded
0
Triple X([43565] CIBBL - Year Two, Autumn - Bronze Division, [11713837] Carol I, Match(3936136))
awarded
0
Target Eliminated([43565] CIBBL - Year Two, Autumn - Bronze Division, [11713837] Carol I, Match(3931160), 1, 2, [11705502] Lir